## Plan transfers between areas

This notebook shows how we can use Decision Optimization to optimize the move of affected people between hospitals to avoid being overcapacity.

The problem is solved per department.

There are 5 parts in this notebooks:
1. Load the data form different places (departements, current situation, etc)
2. represent the current situation on a map
3. predict new cases to come for each department
4. plan transfers 
5. display all the transfers from the solution

**DISCLAIMER: this notebook is thought to demonstrate through an example how Machine Learning and Decision Optmization could be used, but it is partially based on fake data and knowledge of the real problem, and some significant work would be required to incorporate real data and real constraints and obejctives so that the outcome could be useful.**

In [5]:
import brunel
import pandas as pd
import math

### 1. Load the data

Let's import data on hospital status (from the `data.gouf.fr` site).

See https://www.data.gouv.fr/fr/datasets/donnees-relatives-a-lepidemie-de-covid-19/#_

In [6]:
url ="https://www.data.gouv.fr/fr/datasets/r/63352e38-d353-4b54-bfd1-f1b3ee1cabd7"
df=pd.read_csv(url, sep=";")
df.head()

,dep,sexe,jour,hosp,rea,rad,dc
0,01,0,2020-03-18,2,0,1,0
1,01,1,2020-03-18,1,0,1,0
2,01,2,2020-03-18,1,0,0,0
3,02,0,2020-03-18,41,10,18,11
4,02,1,2020-03-18,19,4,11,6


Let's remove some departments which are not concerned by possible moves.

In [7]:
deps = df.dep.unique()
deps = deps[deps != '971']
deps = deps[deps != '972']
deps = deps[deps != '973']
deps = deps[deps != '974']
deps = deps[deps != '976']
deps = deps[~pd.isna(deps)]

Let's create a dictionnary of aggregated data per department.

In [8]:
hosp_data = {}
for d in deps:
    data = df[df.dep==d]
    data = data.groupby('jour').sum().reset_index().drop(columns=['sexe'])
    data['jour']=pd.to_datetime(data['jour'])
    data = data.sort_values(by=['jour'], ascending=False)
    hosp_data[d] = data    

We will use some GPS data on departments from a loaded data asset.

In [9]:
# The code was removed by Watson Studio for sharing.

,Nd,DEPARTEMENT,CAPACITY,LONGITUDE,LATITUDE
0,01,AIN,26,"5d20'56"" E","46d05'58"""
1,02,AISNE,43,"3d33'30"" E","49d33'34"""
2,03,ALLIER,33,"3d11'18"" E","46d23'37"""
3,04,ALPES-DE-HAUTE-PROVENCE,13,"6d14'38"" E","44d06'22"""
4,05,HAUTES-ALPES,24,"6d15'47"" E","44d39'49"""


Lets fix coordinates from string to float values.

In [10]:
def coord(val):
    neg = (val[-1] == 'O')
    #print (pos)
    c1 = val.split('d')[0]
    #print (c1)
    c2 = val.split('d')[1].split("'")[0]
    #print (c2)
    c3 = val.split('d')[1].split("'")[1].split('"')[0]
    #print (c3)
    res = float(c1) + float(c2)/60 + float(c3)/3600
    if neg:
        res = -res
    return res
    

deps_data['LONGITUDE'] = deps_data['LONGITUDE'].apply(lambda x: coord(x))
deps_data['LATITUDE'] = deps_data['LATITUDE'].apply(lambda x: coord(x))

deps_data = deps_data.set_index(['Nd'])
deps_data.head()

,DEPARTEMENT,CAPACITY,LONGITUDE,LATITUDE
Nd,,,,
01,AIN,26,5.348889,46.099444
02,AISNE,43,3.558333,49.559444
03,ALLIER,33,3.188333,46.393611
04,ALPES-DE-HAUTE-PROVENCE,13,6.243889,44.106111
05,HAUTES-ALPES,24,6.263056,44.663611


In [11]:
def distance(lt1, lg1, lt2, lg2):

    R = 6373.0
    
    lat1 = math.radians(lt1)
    lon1 = math.radians(lg1)
    lat2 = math.radians(lt2)
    lon2 = math.radians(lg2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    distance = R * c
    return distance
    

### 2. Display the current situation on a map

We will use folium to represent the data on maps.

In [12]:
!pip install folium

     |████████████████████████████████| 92kB 6.8MB/s eta 0:00:011


Let's now represent the situation for each department

In [13]:
import folium 

m = folium.Map(
    location=[45.5236, 5.6750],
    zoom_start=6
)

initial = {}
capacity = {}

for d, dep in deps_data.iterrows():    
    sz = int(hosp_data[d]['rea'].iat[0])
    initial[d] = sz
    capacity[d] = int(1.2*dep['CAPACITY'])
    
url = 'https://france-geojson.gregoiredavid.fr/repo/departements.geojson'

def style_function(feature):
    d = feature['properties']['code']
    return {
        'fillOpacity': 0.4 if initial[d]>capacity[d] else 0,
        'weight': 0.5,
        'color': 'black',
        'fillColor': 'red' if initial[d]>capacity[d] else 'white'
    }

folium.GeoJson(
    url,
    name='geojson',
    style_function=style_function
).add_to(m)

for d, dep in deps_data.iterrows():    
    lt = dep['LATITUDE']
    lg = dep['LONGITUDE']
    nm = dep['DEPARTEMENT']
    sz = int(hosp_data[d]['rea'].iat[0])
    initial[d] = sz
    #folium.Marker([lt, lg]).add_to(here_map)    
    color = 'green'
    if initial[d]>0.5*capacity[d]:
        color = 'orange'
    if initial[d]>capacity[d]:
        color='red'
    folium.Circle(
        radius=sz*50,
        location=[lt, lg],
        popup=folium.Popup(nm + '(' + str(d)+')<br>'+str(sz)+' reas.<br>Capacity: '+str(capacity[d]),max_width=450,min_width=150),
        color=color,
        fill=True,
        fill_color=color
    ).add_to(m)


m

Create dataframe with status before transfers and calculate the total over capacity

In [14]:
before = pd.DataFrame(columns=['Nd', 'capacity', 'value', 'overcapacity'], data=[[d, capacity[d], initial[d], max(0, initial[d]-capacity[d])] for d in deps])
print (before)

total_overall_before = before['overcapacity'].sum()
print(total_overall_before)

    Nd  capacity  value  overcapacity
0   01        31     12             0
1   02        51     70            19
2   03        39      1             0
3   04        15      2             0
4   05        28     12             0
5   06       160     35             0
6   07        22     11             0
7   08        25     14             0
8   09        25      6             0
9   10        19     26             7
10  11        37     22             0
11  12        34     10             0
12  13       296    194             0
13  14       110     32             0
14  15        20      8             0
15  16        27      5             0
16  17        63     22             0
17  18        16     12             0
18  19        22     12             0
19  21        12     80            68
20  22        34      8             0
21  23        69      0             0
22  24        49      8             0
23  25        27     58            31
24  26        48     59            11
25  27      

### 3. Predict the new cases

We use a very dummay linear regresion to predict the new cases to appear on each department

In [15]:
import numpy as np
from sklearn.linear_model import LinearRegression

   
NB_PERIODS = 3

def predict_more(d, n):   
    X = hosp_data[d].index.tolist()
    X.reverse()
    X=np.array(X).reshape(-1,1)
    y = hosp_data[d]['rea'].tolist()
    y.reverse()
    y=np.array(y).reshape(-1,1)

    regsr=LinearRegression()
    regsr.fit(X,y)

    to_predict_x = [i for i in range(len(X), len(X)+n)]
    to_predict_x= np.array(to_predict_x).reshape(-1,1)
    predicted_y= regsr.predict(to_predict_x)
    delta_y = [ int(round(max(0, predicted_y[i][0]-y[len(y)-1][0]))) for i in range(n)]
    return delta_y

new_rea ={ d:predict_more(d, NB_PERIODS) for d in deps}
print (new_rea)

{'01': [1, 2, 4], '02': [3, 9, 15], '03': [0, 0, 0], '04': [0, 0, 0], '05': [0, 0, 2], '06': [3, 7, 11], '07': [0, 1, 2], '08': [0, 0, 0], '09': [1, 2, 2], '10': [0, 0, 2], '11': [3, 3, 4], '12': [0, 0, 0], '13': [20, 42, 64], '14': [0, 4, 8], '15': [0, 1, 2], '16': [3, 4, 5], '17': [0, 0, 1], '18': [0, 0, 1], '19': [0, 0, 1], '21': [11, 17, 22], '22': [0, 0, 0], '23': [0, 0, 0], '24': [2, 3, 4], '25': [10, 16, 22], '26': [2, 8, 15], '27': [0, 1, 3], '28': [0, 4, 8], '29': [0, 0, 0], '2A': [3, 5, 7], '2B': [0, 0, 0], '30': [0, 3, 5], '31': [0, 0, 9], '32': [1, 1, 1], '33': [13, 25, 37], '34': [7, 13, 19], '35': [6, 9, 11], '36': [4, 6, 8], '37': [1, 3, 4], '38': [0, 1, 5], '39': [0, 0, 0], '40': [0, 0, 0], '41': [0, 1, 1], '42': [0, 3, 11], '43': [0, 0, 1], '44': [6, 10, 14], '45': [2, 7, 13], '46': [0, 0, 0], '47': [0, 0, 0], '48': [0, 1, 2], '49': [1, 6, 11], '50': [4, 5, 7], '51': [9, 19, 29], '52': [0, 2, 5], '53': [0, 0, 1], '54': [19, 39, 60], '55': [1, 3, 4], '56': [0, 0, 0], '5

### 4. Optimize the transfers using Decision Optimization

Now let's create an optimization model.

**Not knowing the real capacities and the real constraints and objectives, we have imagined that some limited short and long transfers could be organized in order to minimize the total over capacity.**

In [16]:
from docplex.mp.environment import Environment
env = Environment()
env.print_information()

* system is: Linux 64bit
* Python version 3.6.9, located at: /opt/conda/envs/Python36/bin/python
* docplex is present, version is (2, 12, 182)
* CPLEX library is present, version is 12.9.0.1, located at: /opt/conda/envs/Python36/lib/python3.6/site-packages
* pandas is present, version is 0.24.1


Parameters are the number of possible moves, and the maximum number of cases per move.

In [17]:
from docplex.mp.model import Model
mdl = Model("Plan Transfers")

MAX_NB_LONG_TRANSFERS_PER_PERIOD = 3
MAX_CASES_PER_LONG_TRANSFERS = 20

MAX_NB_SHORT_TRANSFERS_PER_DEPARTMENT = 3

LONG_DISTANCE = 200

In [18]:
is_long = {d1:{d2: distance(deps_data['LATITUDE'][d1], deps_data['LONGITUDE'][d1], deps_data['LATITUDE'][d2], deps_data['LONGITUDE'][d2]) > LONG_DISTANCE for d2 in deps} for d1 in deps}

In [19]:
transfer_periods = list(range(NB_PERIODS))
all_periods = list(range(NB_PERIODS+1))

# Binary vars representing the "assigned" libraries for each coffee shop
use_link_vars = mdl.binary_var_cube(deps, deps, transfer_periods, name="use_link")
link_vars = mdl.integer_var_cube(deps, deps, transfer_periods, lb=0, ub=MAX_CASES_PER_LONG_TRANSFERS, name="link")
occupancy_vars = mdl.integer_var_matrix(deps, all_periods, lb=0, name="occupancy")

# Short transfers bounds
mdl.add_constraints(link_vars[d1, d2, t] <= 1 for d1 in deps for d2 in deps if not is_long[d1][d2] for t in transfer_periods)

mdl.print_information()

Model: Plan Transfers
 - number of variables: 55680
   - binary=27648, integer=28032, continuous=0
 - number of constraints: 4890
   - linear=4890
 - parameters: defaults
 - problem type is: MILP


Add the contraints 


In [20]:
# Initial state
mdl.add_constraints(occupancy_vars[d, 0] == initial[d] for d in deps)

# structural constraint between user_link and link
mdl.add_constraints(use_link_vars[d, d1, t] == (link_vars[d, d1, t] >= 1) for d in deps for d1 in deps for t in transfer_periods)

# number of transfers from a department less than current number of cases
mdl.add_constraints(mdl.sum(link_vars[d, d1, t] for d1 in deps) <= occupancy_vars[d, t] for d in deps for t in transfer_periods)

# maximum number of LONG transfers
mdl.add_constraints(mdl.sum(use_link_vars[d1, d2, t] for d1 in deps for d2 in deps if is_long[d1][d2]) <= MAX_NB_LONG_TRANSFERS_PER_PERIOD for t in transfer_periods)


# maximum number of SHORT transfers
mdl.add_constraints(mdl.sum(use_link_vars[d1, d2, t] for d1 in deps if not is_long[d1][d2] for t in transfer_periods) <= MAX_NB_SHORT_TRANSFERS_PER_DEPARTMENT for d2 in deps )

# conservation constraints including new cases to come
mdl.add_constraints(occupancy_vars[d, t+1] == new_rea[d][t] + occupancy_vars[d, t] + mdl.sum(link_vars[d1, d, t] for d1 in deps) - mdl.sum(link_vars[d, d1, t] for d1 in deps) for d in deps for t in transfer_periods)

mdl.print_information()

Model: Plan Transfers
 - number of variables: 83328
   - binary=55296, integer=28032, continuous=0
 - number of constraints: 60957
   - linear=33309, equiv=27648
 - parameters: defaults
 - problem type is: MILP


Objective is to minimize the total over capacity  at the end

In [21]:
final_overcapacity = mdl.sum(mdl.max(0,  occupancy_vars[d, NB_PERIODS] - capacity[d]) for d in deps)
mdl.add_kpi(final_overcapacity)

nb_long_transfers = mdl.sum(use_link_vars[d1, d2, t] for d1 in deps for d2 in deps if is_long[d1][d2] for t in transfer_periods)
mdl.add_kpi(nb_long_transfers)

nb_short_transfers = mdl.sum(use_link_vars[d1, d2, t] for d1 in deps for d2 in deps if not is_long[d1][d2] for t in transfer_periods)
mdl.add_kpi(nb_short_transfers)

mdl.minimize(1000 * final_overcapacity + 10 * nb_long_transfers + nb_short_transfers)

Solve the problem

In [22]:
mdl.set_time_limit(20) 
mdl.solve(log_output=True)

CPXPARAM_Read_DataCheck                          1
CPXPARAM_Threads                                 2
CPXPARAM_TimeLimit                               20
Tried aggregator 2 times.
MIP Presolve eliminated 5470 rows and 1054 columns.
Aggregator did 32691 substitutions.
Reduced MIP has 45785 rows, 50255 columns, and 199671 nonzeros.
Reduced MIP has 27266 binaries, 22797 generals, 0 SOSs, and 21269 indicators.
Presolve time = 0.55 sec. (216.15 ticks)
Probing fixed 0 vars, tightened 192 bounds.
Probing time = 0.44 sec. (72.99 ticks)
Cover probing fixed 0 vars, tightened 294 bounds.
Tried aggregator 1 time.
Reduced MIP has 45787 rows, 50255 columns, and 199675 nonzeros.
Reduced MIP has 27266 binaries, 22797 generals, 0 SOSs, and 21267 indicators.
Presolve time = 0.43 sec. (164.32 ticks)
Probing time = 0.03 sec. (11.10 ticks)
Clique table members: 1.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 2 threads.
Root r

docplex.mp.solution.SolveSolution(obj=3.23725e+06,values={use_link_02_06..

Extract and print the moves

In [23]:
edges = [(t, d1, d2, int(link_vars[d1, d2, t]), is_long[d1][d2]) for t in transfer_periods for d1 in deps for d2 in deps if int(link_vars[d1, d2, t]) >= 1]
print (edges)

[(0, '02', '06', 20, True), (0, '02', '08', 1, False), (0, '02', '27', 1, False), (0, '02', '59', 1, False), (0, '02', '62', 1, False), (0, '02', '75', 1, False), (0, '02', '76', 1, False), (0, '09', '32', 1, False), (0, '09', '47', 1, False), (0, '09', '64', 1, False), (0, '10', '18', 1, False), (0, '10', '58', 1, False), (0, '13', '04', 1, False), (0, '13', '05', 1, False), (0, '13', '06', 1, False), (0, '13', '30', 1, False), (0, '13', '34', 1, False), (0, '13', '38', 1, False), (0, '13', '48', 1, False), (0, '13', '84', 1, False), (0, '21', '01', 1, False), (0, '21', '03', 1, False), (0, '21', '39', 1, False), (0, '21', '42', 1, False), (0, '25', '73', 1, False), (0, '26', '07', 1, False), (0, '26', '43', 1, False), (0, '26', '63', 1, False), (0, '28', '14', 1, False), (0, '28', '36', 1, False), (0, '28', '37', 1, False), (0, '28', '41', 1, False), (0, '28', '49', 1, False), (0, '28', '53', 1, False), (0, '28', '61', 1, False), (0, '28', '72', 1, False), (0, '29', '22', 1, False), 

Recreate structure for final situation, and calculate final over capacity.

In [24]:
final = { d: int(occupancy_vars[d, NB_PERIODS].solution_value) for d in deps }

after = pd.DataFrame(columns=['Nd', 'capacity', 'value', 'overcapacity'], data=[[d, capacity[d], final[d], max(0, final[d]-capacity[d])] for d in deps])
print(after)

total_overall_after = after['overcapacity'].sum()
print(total_overall_after)

    Nd  capacity  value  overcapacity
0   01        31     22             0
1   02        51     59             8
2   03        39     39             0
3   04        15      5             0
4   05        28     17             0
5   06       160    159             0
6   07        22     17             0
7   08        25     17             0
8   09        25     25             0
9   10        19     22             3
10  11        37     35             0
11  12        34     33             0
12  13       296    296             0
13  14       110     47             0
14  15        20     14             0
15  16        27     20             0
16  17        63     23             0
17  18        16     16             0
18  19        22     16             0
19  21        12    118           106
20  22        34     11             0
21  23        69      3             0
22  24        49     20             0
23  25        27    103            76
24  26        48     55             7
25  27      

### 5. Display the solution

And represent the moves and the changes for impacted departments.

* Black links are the local transfers.
* Green links are the long distance transfers.

In [25]:
# The code was removed by Watson Studio for sharing.

In [26]:

m = folium.Map(
    location=[45.5236, 5.6750],
    zoom_start=6
)

    
url = 'https://france-geojson.gregoiredavid.fr/repo/departements.geojson'

def style_function(feature):
    d = feature['properties']['code']
    return {
        'fillOpacity': 0.4 if final[d]>capacity[d] else 0,
        'weight': 0.5,
        'color': 'black',
        'fillColor': 'red' if final[d]>capacity[d] else 'white'
    }

folium.GeoJson(
    url,
    name='geojson',
    style_function=style_function
).add_to(m)


for d, dep in deps_data.iterrows():    
    if final[d]==initial[d]:
        continue
    lt = dep['LATITUDE']
    lg = dep['LONGITUDE']
    nm = dep['DEPARTEMENT']
    sz0 = initial[d]
    sz1 = final[d]
    color = 'green'
    if final[d]>0.5*capacity[d]:
        color = 'orange'
    if final[d]>capacity[d]:
        color='red'    
    folium.Circle(
        radius=sz1*50,
        location=[lt, lg],
        popup=folium.Popup(nm+' (' + str(d)+'):<br>From '+str(sz0)+' to '+str(sz1)+' reas.<br>Capacity: '+str(capacity[d]),max_width=450,min_width=150),
        color=color,
        fill=True,
        fill_color=color
    ).add_to(m)
        
for (t, c, b, v, il) in edges:            
    ic = deps_data.index.get_loc(c)
    ib = deps_data.index.get_loc(b)
    dep1 = deps_data.iloc[ic]
    dep2 = deps_data.iloc[ib]
    coordinates = [[dep1['LATITUDE'], dep1['LONGITUDE']], [dep2['LATITUDE'], dep2['LONGITUDE']]]
    color = 'green'
    weight = 4
    if not il:
        color = 'black'
        weight = 2
    pl = folium.PolyLine(coordinates, color=color, weight=weight)
    m.add_child(pl)
    
    arrows = get_arrows(locations=coordinates, color=color, size=3, n_arrows=3)
    for arrow in arrows:
        arrow.add_to(m)
    
    
    
m